In [1]:
import json
from urllib.parse import quote
import pandas as pd
import requests
from config import DATABASE_ShOWS_ID, TOKEN_V3, TRAKT_CLIENT_ID, TMDB_API_KEY
from notion_client import Client


In [23]:
with open('notion_seriedb_json_example.json', 'r', encoding='utf-8-sig') as file:
    request = json.load(file)
    dados = request[0]['data']

if dados.get("object") == "page":
    page_id = dados.get("id")
    title = dados['properties']['Name']['title'][0]['plain_text']
    year = dados['properties']['Year']['rich_text'][0]['plain_text']
    title_url = title.replace(" ", "-").lower()

title

'The Boys'

In [ ]:
 # Obter dados do show do TMDB
tmdb_url_show = f"https://api.themoviedb.org/3/tv/{show_id}"
tmdb_url_show = f"https://api.themoviedb.org/3/search/tv?query={title}&first_air_date_year={year}&page=1"
headers = {
    "accept": "application/json",
    "authorization": f"Bearer {TMDB_API_KEY}"
}
response = requests.get(tmdb_url_show, headers=headers)
show_data = response.json()

print(show_data['results'][0]['id'])


{'adult': False, 'backdrop_path': '/7cqKGQMnNabzOpi7qaIgZvQ7NGV.jpg', 'genre_ids': [10765, 10759], 'id': 76479, 'origin_country': ['US'], 'original_language': 'en', 'original_name': 'The Boys', 'overview': 'A group of vigilantes known informally as “The Boys” set out to take down corrupt superheroes with no more than blue-collar grit and a willingness to fight dirty.', 'popularity': 102.2605, 'poster_path': '/2zmTngn1tYC1AvfnrFLhxeD82hz.jpg', 'first_air_date': '2019-07-25', 'name': 'The Boys', 'vote_average': 8.5, 'vote_count': 11086}


In [29]:

# title = 'The Office'
def get_show_id(title, TRAKT_CLIENT_ID):
    trakt_url = f"https://api.trakt.tv/shows/{quote(title)}?extended=full"
    headers = {
        "Content-Type": "application/json",
        "trakt-api-version": "2",
        "trakt-api-key": TRAKT_CLIENT_ID
    }
    response = requests.get(trakt_url, headers=headers)
    data = response.json()
    show_id = data['ids']['tmdb']
    print(trakt_url)
    return show_id

# Example usage
show_id = get_show_id(title, TRAKT_CLIENT_ID)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
# def get_show_data(page_id, TRAKT_CLIENT_ID):
tmdb_url_show = f"https://api.themoviedb.org/3/tv/{show_id}"
headers = {
    "accept": "application/json",
    "authorization": f"Bearer {TMDB_API_KEY}"
}
response = requests.get(tmdb_url_show, headers=headers)
show_data = response.json()
print(show_data)

networks = show_data['networks'][0]['name']
first_air_date = show_data['first_air_date']
last_air_date = show_data['last_air_date']
poster = "https://image.tmdb.org/t/p/original" + show_data['poster_path']
Date = pd.Timestamp.today().strftime('%Y-%m-%d')
Overview = show_data['overview']
episode_count = show_data['number_of_episodes']
seasons_count = show_data['number_of_seasons']
series_status = show_data['status']
if show_data['next_episode_to_air']:
    next_episode_to_air = show_data['next_episode_to_air']['air_date'] if show_data['next_episode_to_air'] else None


{'adult': False, 'backdrop_path': '/wQEW3xLrQAThu1GvqpsKQyejrYS.jpg', 'created_by': [{'id': 2000007, 'credit_id': '62d5e468c92c5d004f0d1201', 'name': 'Christian Linke', 'original_name': 'Christian Linke', 'gender': 2, 'profile_path': '/u1uCJkmVSeLQC1CwSB7UzXAVbqO.jpg'}, {'id': 3299121, 'credit_id': '62d5e46e72c13e062e7196aa', 'name': 'Alex Yee', 'original_name': 'Alex Yee', 'gender': 2, 'profile_path': '/kvRKbXZcCnM6d1vcND4917iQ4kp.jpg'}], 'episode_run_time': [], 'first_air_date': '2021-11-06', 'genres': [{'id': 16, 'name': 'Animation'}, {'id': 10765, 'name': 'Sci-Fi & Fantasy'}, {'id': 18, 'name': 'Drama'}, {'id': 10759, 'name': 'Action & Adventure'}], 'homepage': 'https://arcane.com', 'id': 94605, 'in_production': False, 'languages': ['en'], 'last_air_date': '2024-11-23', 'last_episode_to_air': {'id': 5609651, 'name': 'The Dirt Under Your Nails', 'overview': 'Magic. Science. Power. Revenge. Destinies clash in an epic final chapter, igniting an all-out war.', 'vote_average': 8.2, 'vot

In [5]:
client = Client(auth=TOKEN_V3)
database_id = DATABASE_ShOWS_ID

response = client.pages.update(
    page_id=page_id,
    properties={
        "Name": {"title": [{"text": {"content": title}}]},
        "Release Date": {"date": {"start": first_air_date, "end": last_air_date}},
        "Poster": {"files": [{"name": "Poster", "external": {"url": poster}}]},
        "Episode Count": {"number": episode_count},
        "Network": {"select": {"name": networks}},
        "Overview": {"rich_text": [{"text": {"content": Overview}}]},
        "Seasons Count": {"number": seasons_count},
        "Type": {"select": {"name": "Show"}},
        "Series Status": {"select": {"name": series_status}},
    }
)


In [6]:

for season in show_data['seasons']:
    if season['name'] != 'Specials':
        season_name = season['name']
        season_number = season['season_number']
        season_id = season['id']
        release_date = season['air_date']
        episode_count = season['episode_count']
        season_overview = season['overview']
        poster = "https://image.tmdb.org/t/p/original" + season['poster_path']
        response = client.pages.create(
            parent={"database_id": database_id},
            properties={
                "Name": {"title": [{"text": {"content": title + f" - Season {season_number}"}}]},
                "Release Date": {"date": {"start": release_date}},
                "Season Number": {"number": season_number},
                "Episode Count": {"number": episode_count},
                "Overview": {"rich_text": [{"text": {"content": season_overview}}]},
                "Type": {"select": {"name": "Season"}},
                "Poster": {"files": [{"name": "Poster", "external": {"url": poster}}]},
                "Network": {"select": {"name": networks}},
                "Parent item": {"relation": [{"id": page_id}]},
            }
        )
        season_page_id = response['id']
        tmdb_url_episode = f"https://api.themoviedb.org/3/tv/{show_id}/season/{season_number}"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {TMDB_API_KEY}"
        }
        response = requests.get(tmdb_url_episode, headers=headers)
        eps_data = response.json()
        print(eps_data)

        for episode in eps_data['episodes']:
            episode_name = str(episode['episode_number']) + " - " + episode['name']
            episode_number = episode['episode_number']
            season_number = episode['season_number']
            release_date = episode['air_date']
            episode_overview = episode['overview']
            poster = "https://image.tmdb.org/t/p/original" + episode['still_path']
            response = client.pages.create(
                parent={"database_id": database_id},
                properties={
                    "Name": {"title": [{"text": {"content": episode_name}}]},
                    "Release Date": {"date": {"start": release_date}},
                    "Episode Number": {"number": episode_number},
                    "Season Number": {"number": season_number},
                    "Overview": {"rich_text": [{"text": {"content": episode_overview}}]},
                    "Type": {"select": {"name": "Episode"}},
                    "Poster": {"files": [{"name": "Poster", "external": {"url": poster}}]},
                    "Parent item": {"relation": [{"id": season_page_id}]},
                    "Network": {"select": {"name": networks}},
                }
            )



{'_id': '5da6ba97a2423200127893a2', 'air_date': '2021-11-06', 'episodes': [{'air_date': '2021-11-06', 'episode_number': 1, 'episode_type': 'standard', 'id': 1953812, 'name': 'Welcome to the Playground', 'overview': "Orphaned sisters Vi and Powder bring trouble to Zaun's underground streets in the wake of a heist in posh Piltover.", 'production_code': '', 'runtime': 44, 'season_number': 1, 'show_id': 94605, 'still_path': '/g7dMkFJvJECDYoL0TRzGQCfF3xD.jpg', 'vote_average': 8.355, 'vote_count': 124, 'crew': [{'job': 'Animation', 'department': 'Visual Effects', 'credit_id': '61a7b438ed28b90024d9ec04', 'adult': False, 'gender': 2, 'id': 2530723, 'known_for_department': 'Visual Effects', 'name': 'Martin Touzé', 'original_name': 'Martin Touzé', 'popularity': 0.001, 'profile_path': None}, {'job': 'Writer', 'department': 'Writing', 'credit_id': '61d4f4d477c01f0067beea3c', 'adult': False, 'gender': 2, 'id': 3299121, 'known_for_department': 'Production', 'name': 'Alex Yee', 'original_name': 'Alex